In [1]:
import requests as rq
import pandas as pd

In [2]:
base_url = 'https://swapi.dev/api/people/'

In [3]:
data = rq.get(f'{base_url}10/').json()
character_fields = data.keys()
print(*character_fields, sep='\n')

name
height
mass
hair_color
skin_color
eye_color
birth_year
gender
homeworld
films
species
vehicles
starships
created
edited
url


In [5]:
rq.get(f'{base_url}10/').json()

{'name': 'Obi-Wan Kenobi',
 'height': '182',
 'mass': '77',
 'hair_color': 'auburn, white',
 'skin_color': 'fair',
 'eye_color': 'blue-gray',
 'birth_year': '57BBY',
 'gender': 'male',
 'homeworld': 'https://swapi.dev/api/planets/20/',
 'films': ['https://swapi.dev/api/films/1/',
  'https://swapi.dev/api/films/2/',
  'https://swapi.dev/api/films/3/',
  'https://swapi.dev/api/films/4/',
  'https://swapi.dev/api/films/5/',
  'https://swapi.dev/api/films/6/'],
 'species': [],
 'vehicles': ['https://swapi.dev/api/vehicles/38/'],
 'starships': ['https://swapi.dev/api/starships/48/',
  'https://swapi.dev/api/starships/59/',
  'https://swapi.dev/api/starships/64/',
  'https://swapi.dev/api/starships/65/',
  'https://swapi.dev/api/starships/74/'],
 'created': '2014-12-10T16:16:29.192000Z',
 'edited': '2014-12-20T21:17:50.325000Z',
 'url': 'https://swapi.dev/api/people/10/'}

In [2]:
def get_character(id):
    base_url = 'https://swapi.dev/api/people'
    try:
        content = rq.get(f'{base_url}/{id}').json()
    except Exception as e:
        print('URL not found!')
        return
    
    referred_fields = ['homeworld', 'films', 'species', 'vehicles', 'starships']

    for field in referred_fields:
        id_values = []
        try:
            if content[field]:
                if field != 'homeworld':
                    for link in content[field]:
                        id_values.append(int(link.split('/')[-2]))
                    content[field] = id_values
                # homeworld is a single url string
                else:
                    content[field] = int(content[field].split('/')[-2])
            elif field == 'species' and not content[field]:
                content[field] = 'human'
        except:
            print(f"{field} doesn't exist in character: {base_url}/{id}")
            pass
             
    # remove created and edited fields
    try:
        del(content['created'])
        del(content['edited'])
    except:
        print(f'Keys not found in content! ({base_url}/{id})')

    return content

In [3]:
get_character(10)

{'name': 'Obi-Wan Kenobi',
 'height': '182',
 'mass': '77',
 'hair_color': 'auburn, white',
 'skin_color': 'fair',
 'eye_color': 'blue-gray',
 'birth_year': '57BBY',
 'gender': 'male',
 'homeworld': 20,
 'films': [1, 2, 3, 4, 5, 6],
 'species': 'human',
 'vehicles': [38],
 'starships': [48, 59, 64, 65, 74],
 'url': 'https://swapi.dev/api/people/10/'}

In [4]:
characters_list = [get_character(id) for id in range(1,83)]

homeworld doesn't exist in character: https://swapi.dev/api/people/17
films doesn't exist in character: https://swapi.dev/api/people/17
species doesn't exist in character: https://swapi.dev/api/people/17
vehicles doesn't exist in character: https://swapi.dev/api/people/17
starships doesn't exist in character: https://swapi.dev/api/people/17
Keys not found in content! (https://swapi.dev/api/people/17)


In [19]:
for index, value in enumerate(characters_list):
    if 'detail' in value.keys():
        print(f'Details found in character with id: {index + 1}')

Details found in character with id: 17


Remove character with id 17. It's a valid url!

In [27]:
del characters_list[16]

In [28]:
df = pd.DataFrame(characters_list)

In [30]:
df.head()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,url
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,1,"[1, 2, 3, 6]",human,"[14, 30]","[12, 22]",https://swapi.dev/api/people/1/
1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,1,"[1, 2, 3, 4, 5, 6]",[2],[],[],https://swapi.dev/api/people/2/
2,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,8,"[1, 2, 3, 4, 5, 6]",[2],[],[],https://swapi.dev/api/people/3/
3,Darth Vader,202,136,none,white,yellow,41.9BBY,male,1,"[1, 2, 3, 6]",human,[],[13],https://swapi.dev/api/people/4/
4,Leia Organa,150,49,brown,light,brown,19BBY,female,2,"[1, 2, 3, 6]",human,[30],[],https://swapi.dev/api/people/5/


In [31]:
df.shape

(81, 14)

In [32]:
df.to_csv('sw_characters.csv')